In [1]:
import sys
import os
sys.path.append('./DBmanipulation')
sys.path.append('../DBmanipulation')
sys.path.append('./AIMemory')
sys.path.append('../AIMemory')

import DBCon 
import BufferDB
import InstructionDB
import ReflectionTracerDB
import MemStreDB
import ReflectionDB
# import ScheduleDB

import GPTProcess
import ManualProcess
import InstToMemStre
import InputToMemStre



import pandas as pd
import numpy as np
import json
import re
import pickle


True
Current working directory: /Users/jackhan/Desktop/AITown/satoshiLive/AIModule
Config sections found: ['mysql', 'Paths']
Connected to MySQL server and database 'AITown'


In [2]:

instruction_db_conn = schedule_db_conn = buffer_db_conn = reflection_tracer_db_conn = reflection_db_conne = memstre_db_conn = buffer_db_conn= DBCon.establish_sql_connection()
input_from_java = BufferDB.get_earliest_unprocessed_entry(buffer_db_conn)
print(input_from_java)

if input_from_java is None:
    print('Nothing to process so far')
    print()
    print()




npcId = input_from_java[1]
AllEntryOfNPC, LatestEntryOfNpc = BufferDB.get_unprocessed_entries_of_npc(buffer_db_conn, npcId)
input_from_java = LatestEntryOfNpc



curTime = input_from_java[0]
inputInHumanString = ManualProcess.parse_npc_info(input_from_java[2])


Current working directory: /Users/jackhan/Desktop/AITown/satoshiLive/AIModule
Config sections found: ['mysql', 'Paths']
Connected to MySQL server and database 'AITown'
Earliest unprocessed entry: time=2025-02-05 14:29:03, npcId=10006, content={"world": {"time": 1738736943494}, "mapObj": [{"oid": "zhongbencongType", "objName": "zhongbencongType", "type": "", "X": 463, "Y": 722, "status": "DEFAULT", "availableActions": []}, {"oid": "zhongbencongRead", "objName": "zhongbencongRead", "type": "", "X": 240, "Y": 688, "status": "DEFAULT", "availableActions": []}, {"oid": "zhongbencongFix", "objName": "zhongbencongFix", "type": "", "X": 370, "Y": 304, "status": "DEFAULT", "availableActions": []}, {"oid": "zhongbencongThink", "objName": "zhongbencongThink", "type": "", "X": 336, "Y": 721, "status": "DEFAULT", "availableActions": []}], "npcs": [{"npcId": 10006, "status": "idle", "info": {"name": "wSatoshi", "firstName": "wSatoshi", "lastName": "wSatoshi", "type": "Engineer", "age": 20, "height":

In [3]:
# Get Relavent Memeory from Memeory Stream
BufferRowEmbedding = GPTProcess.get_embedding(inputInHumanString)
rows_df = MemStreDB.retrieve_most_recent_entries(memstre_db_conn, npcId, curTime)
if rows_df is not None:
    rows_df['Time'] = pd.to_datetime(rows_df['Time'])
    rows_df['TimeDifference'] = (rows_df['Time'] - pd.to_datetime(curTime)).dt.total_seconds()
    decay_rate = 0.001 
    rows_df['recency'] = np.exp(decay_rate * rows_df['TimeDifference'])
    def cosine_similarity(vec1, vec2):
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        return dot_product / (norm_vec1 * norm_vec2)

    rows_df['cosine_similarity'] = rows_df['Embedding'].apply(lambda x: cosine_similarity(BufferRowEmbedding, np.array(x)))

    a_recency = 0.5  # Adjust the weight for recency as needed
    a_importance = 0.3  # Adjust the weight for importance as needed
    a_similarity = 0.2  # Adjust the weight for similarity as needed


    rows_df['retrieval_score'] = (
        a_recency * rows_df['recency'] +
        a_importance * rows_df['Importance'] + 
        a_similarity * rows_df['cosine_similarity']
    )

    rows_df_ranked = rows_df.sort_values(by=['retrieval_score', 'Time'], ascending=[False, False]).head(50)
    rows_df_ranked = rows_df_ranked.sort_values(by='Time', ascending=False)
    rows_df_ranked
    memories_str = str(list(rows_df_ranked['Content']))
else:
    memories_str = ''



#Retrive latest reflection
prior_reflection = ReflectionDB.retrieve_last_entry_before_time(reflection_db_conne, npcId, curTime)
if prior_reflection is not None:
    prior_reflection_str = str(prior_reflection[2])
else:
    prior_reflection_str = ''




Retrieved 0 entries for npcID=10006 before time=2025-02-05 14:29:03
No entries found for npcID=10006 before 2025-02-05 14:29:03


In [4]:
inputInHumanString

'Time: 2025-02-05 14:29:03\nNPC First Name: wSatoshi\nNPC Last Name: wSatoshi\nNPC Learned: wSatoshi is a hardworking researcher and engineer who reading, thinking, and using computer all day long.\nNPC Lifestyle: wSatoshi spends his days reading, typing, thinking, and repairing the Coinnie, and he enjoy going out to meet other characters.\nNPC Living Area: house\nNPC Current Action: Reading'

In [5]:

# Process these information 
instruction_in_human = GPTProcess.process_all_give_conversation(memories_str, prior_reflection_str, inputInHumanString, 'be dramatic')

In [6]:
instruction_in_human

'As I delve deeper into this mysterious text, a thrilling thought ignites within me… What if the secrets of the universe lie hidden within these pages? I must push forward, but I yearn for companionship! Who will join me on this grand intellectual adventure?'

In [7]:

# Translate to instruction
instruction_to_give = ManualProcess.talkingInstruction(npcId, instruction_in_human)

In [8]:
instruction_to_give

'\n    {\n        "actionId": 110,\n        "npcId": "10006",\n        "data": {\n            "content": "As I delve deeper into this mysterious text, a thrilling thought ignites within me… What if the secrets of the universe lie hidden within these pages? I must push forward, but I yearn for companionship! Who will join me on this grand intellectual adventure?"\n        }\n    }\n    '

In [9]:



# Mark the buffer as processed
BufferDB.mark_entry_as_processed(buffer_db_conn, curTime, npcId)
for row in AllEntryOfNPC:
    time_to_mark = row[0]
    npcId_to_mark = row[1]
    BufferDB.mark_entry_as_processed(buffer_db_conn, time_to_mark, npcId_to_mark)
# add to instruction db
InstructionDB.insert_into_instruction_table(instruction_db_conn, curTime, npcId, instruction_to_give)

# insert instruction to Memory Stream
InstToMemStre.InstToMemStreSatoshiDB(input_from_java, instruction_in_human)
# add importance to Reflection Tracer
InstToMemStre.InstImportancetoReflectionTracer( input_from_java, instruction_to_give)
print(instruction_to_give)


Entry with time=2025-02-05 14:29:03 and npcId=10006 marked as processed.
Entry with time=2025-02-05 14:29:03 and npcId=10006 marked as processed.
Instruction inserted successfully: time=2025-02-05 14:29:03, npcId=10006, instruction length=379, isProcessed=False
Data inserted successfully: npcID=10006, time=2025-02-05 14:29:03, isInstruction=1, content length=257, importance=7
Current working directory: /Users/jackhan/Desktop/AITown/satoshiLive/AIModule
Config sections found: ['mysql', 'Paths']
Connected to MySQL server and database 'AITown'
Table 'reflection_tracer' exists in database 'AITown'.
(datetime.datetime(2025, 2, 5, 14, 29, 3), 10006, '{"world": {"time": 1738736943494}, "mapObj": [{"oid": "zhongbencongType", "objName": "zhongbencongType", "type": "", "X": 463, "Y": 722, "status": "DEFAULT", "availableActions": []}, {"oid": "zhongbencongRead", "objName": "zhongbencongRead", "type": "", "X": 240, "Y": 688, "status": "DEFAULT", "availableActions": []}, {"oid": "zhongbencongFix", 